Author: Salih Jasim
# Title: Predicting Customer Churn in a Music Subscription Service
## Project for the special topics in machine learning course 

First we will import important libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
DATA_PATH = r'C:\Users\Saleh\Desktop\kkbox-churn-prediction-challenge'
# DATA_PATH = r'C:\Users\Olabola\Desktop\kkbox-churn-prediction-challenge'

Load all the csv files into memory information about each csv file is shown from the kaggle competition page: https://www.kaggle.com/c/kkbox-churn-prediction-challenge/data

train.csv
the train set, containing the user ids and whether they have churned.

* msno: user id
* is_churn: This is the target variable. Churn is defined as whether the user did not continue the subscription within 30 days of expiration. is_churn = 1 means churn,is_churn = 0 means renewal.

In [3]:
train_df = pd.read_csv(os.path.join(DATA_PATH, 'train_v2.csv', 'data', 'churn_comp_refresh', 'train_v2.csv'))
train_df.head()

,msno,is_churn
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1


members.csv
user information. Note that not every user in the dataset is available.

* msno
* city
* bd: age. Note: this column has outlier values ranging from -7000 to 2015, please use your judgement.
* gender
* registered_via: registration method
* registration_init_time: format %Y%m%d
* expiration_date: format %Y%m%d, taken as a snapshot at which the member.csv is extracted. Not representing the actual churn behavior.

In [4]:
members_df = pd.read_csv(os.path.join(DATA_PATH, 'members_v3.csv', 'members_v3.csv'))
members_df.head()

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,20110915
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,20110915
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915


transactions.csv
transactions of users up until 3/31/2017.

* msno: user id
* payment_method_id: payment method
* payment_plan_days: length of membership plan in days
* plan_list_price: in New Taiwan Dollar (NTD)
* actual_amount_paid: in New Taiwan Dollar (NTD)
* is_auto_renew
* transaction_date: format %Y%m%d
* membership_expire_date: format %Y%m%d
* is_cancel: whether or not the user canceled the membership in this transaction.

In [5]:
transactions_df = pd.read_csv(os.path.join(DATA_PATH, 'transactions_v2.csv', 'data', 'churn_comp_refresh', 'transactions_v2.csv'))
transactions_df.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,20170131,20170504,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,20150809,20190412,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,20170303,20170422,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30,180,180,1,20170329,20170331,1
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30,99,99,1,20170323,20170423,0


user_logs.csv
daily user logs describing listening behaviors of a user. Data collected until 3/31/2017.

* msno: user id
* date: format %Y%m%d
* num_25: # of songs played less than 25% of the song length
* num_50: # of songs played between 25% to 50% of the song length
* num_75: # of songs played between 50% to 75% of of the song length
* num_985: # of songs played between 75% to 98.5% of the song length
* num_100: # of songs played over 98.5% of the song length
* num_unq: # of unique songs played
* total_secs: total seconds played

In [6]:
user_logs_df = pd.read_csv(os.path.join(DATA_PATH, 'user_logs_v2.csv', 'data', 'churn_comp_refresh', 'user_logs_v2.csv'))
user_logs_df.head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=,20170331,8,4,0,1,21,18,6309.273
1,nTeWW/eOZA/UHKdD5L7DEqKKFTjaAj3ALLPoAWsU8n0=,20170330,2,2,1,0,9,11,2390.699
2,2UqkWXwZbIjs03dHLU9KHJNNEvEkZVzm69f3jCS+uLI=,20170331,52,3,5,3,84,110,23203.337
3,ycwLc+m2O0a85jSLALtr941AaZt9ai8Qwlg9n0Nql5U=,20170331,176,4,2,2,19,191,7100.454
4,EGcbTofOSOkMmQyN1NMLxHEXJ1yV3t/JdhGwQ9wXjnI=,20170331,2,1,0,1,112,93,28401.558


### Merging the 'train' and 'members' dataframes
The train dataframe contains the identifiers (msno) for the customer for which we will predict churn as well as whether they churned.

The members dataframe contains user information.

We will merge the dataframes on the 'msno' column.

In [25]:
members_train_df = pd.merge(left=train_df, right=members_df, how='left', on='msno')
members_train_df.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,13.0,20.0,male,3.0,20131223.0
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,13.0,18.0,male,3.0,20131227.0
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,1.0,0.0,NaN,7.0,20140109.0
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,13.0,35.0,female,7.0,20140125.0


In [26]:
members_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 970960 entries, 0 to 970959
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   msno                    970960 non-null  object 
 1   is_churn                970960 non-null  int64  
 2   city                    860967 non-null  float64
 3   bd                      860967 non-null  float64
 4   gender                  388905 non-null  object 
 5   registered_via          860967 non-null  float64
 6   registration_init_time  860967 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 59.3+ MB


In [27]:
members_train_df.isna().sum()

msno                           0
is_churn                       0
city                      109993
bd                        109993
gender                    582055
registered_via            109993
registration_init_time    109993
dtype: int64

## Missing values 
As a result of merging the two dataframes, there are many missing values in the resulting merged dataframe. This is because some of the users in the train dataframe have no information in the members dataframe. 

This occurs in 109,993 rows or 11% of the rows in the merged dataframe. Since this is such a small amount of the data these rows will be removed.

In [28]:
print(f'The merged dataframe contains: {len(members_train_df)} rows')
print(f'There are {members_train_df.city.isna().sum()} users who do not have member information')
print(f'This represents {members_train_df.city.isna().sum() / len(members_train_df) * 100}% of the data')

The merged dataframe contains: 970960 rows
There are 109993 users who do not have member information
This represents 11.328273049353218% of the data


In [29]:
# There are 109,993 instances that have no members information. These rows will be removed.
members_train_df = members_train_df.dropna(subset=['bd'])

In [30]:
members_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 860967 entries, 0 to 970958
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   msno                    860967 non-null  object 
 1   is_churn                860967 non-null  int64  
 2   city                    860967 non-null  float64
 3   bd                      860967 non-null  float64
 4   gender                  388905 non-null  object 
 5   registered_via          860967 non-null  float64
 6   registration_init_time  860967 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 52.5+ MB


In [31]:
members_train_df.isna().sum()

msno                           0
is_churn                       0
city                           0
bd                             0
gender                    472062
registered_via                 0
registration_init_time         0
dtype: int64

A problem remains that 472062 users do not have gender information at all. This is a problem with the member data. We will solve this by making assigning them a gender called "unspecified".

In [32]:
members_train_df.fillna('unspecified', inplace=True)

After performing this step we have no more missing values in the merged dataframe

In [33]:
members_train_df.isna().sum()

msno                      0
is_churn                  0
city                      0
bd                        0
gender                    0
registered_via            0
registration_init_time    0
dtype: int64

### Merging the 'members_train' dataframe with the transactions dataframe
The transactions dataframe contains information on the transactions of users

We will merge the dataframes on the 'msno' column

The problem is that the transactions dataframe contains many different entries for the same user (one for each transaction). So, before we merge the dataframes we must make sure there is only one row per user in the transactions dataframe

In [34]:
# The transactions dataframe does not have unique msno's:
transactions_df.msno.value_counts()

72gJqt1O31E/WoxAEYFn9LHNI6mAZFGera5Q6gvsFkA=    208
5ty4nZkq54z93wQtBN7RHVYj8rNghBDCVBH+3xmxf0I=    172
OGKDrZQDB3yewZhoSd5qqvmG5A1GcNTYMexO95NlH+g=    148
WHsCtkOVsauvqBL0ULuG38887y7aU8GXdCmJMjw6hjQ=    145
SNlFRAsmUqnXKPofSXA8WYUc5DtmLcUMy4pXSJ3Ohz0=    131
                                               ... 
Xg8JXBwcn5S/KPRHrqirNka+NbpRSamJws4IXl65fmQ=      1
r0xassVCM4z5xCe9M5SK/RDC8GN3K9mt1y8HLkGvcFM=      1
x653+WYB2xCA0B8aad7ajCDGOb57LU3KtRDV34ZnMxM=      1
qnErpEifrsMNuZooCCf+Os2R/M1tUkzpwubWbS6SQf0=      1
ySgrbAPphP1Yz1DcBKXCRqR4NXIyUl4Qkb8PknDQtkE=      1
Name: msno, Length: 1197050, dtype: int64

We will create a dataframe called 'transactions_df_unique_msno' that aggregates information for users across multiple transactions, then we will merge is with members_train_df

In [121]:
transactions_df_unique_msno = pd.DataFrame()

The following operations will aggregate user information into one row per user

In [122]:
# Replace the payment method with the most frequent one:
transactions_df_unique_msno['most_frequent_payment_method_id'] = transactions_df.groupby('msno')['payment_method_id'].agg(lambda x: pd.Series.mode(x)[0])

In [123]:
# Add total payment_plan_days column
transactions_df_unique_msno['total_payment_plan_days'] = transactions_df.groupby('msno')['payment_plan_days'].agg(np.sum)

In [124]:
# Add total plan_list_price column
transactions_df_unique_msno['total_plan_list_price'] = transactions_df.groupby('msno')['plan_list_price'].agg(np.sum)

In [125]:
# Add total actual_amount_paid column
transactions_df_unique_msno['total_actual_amount_paid'] = transactions_df.groupby('msno')['actual_amount_paid'].agg(np.sum)

In [126]:
# Add ratio_cancel column
transactions_df_unique_msno['ratio_cancel'] = transactions_df.groupby('msno')['is_cancel'].agg(np.mean)

In [127]:
# Add a ratio_auto_renew column
transactions_df_unique_msno['ratio_auto_renew'] = transactions_df.groupby('msno')['is_auto_renew'].agg(np.mean)

In [128]:
transactions_df_unique_msno.reset_index(inplace=True)

In [87]:
transactions_df_unique_msno.head()

,index,msno,most_frequent_payment_method_id,total_payment_plan_days,total_plan_list_price,total_actual_amount_paid,ratio_cancel,ratio_auto_renew
0,0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,22,395,1599,1599,0.0,0.0
1,1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,0.0,1.0
2,2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,60,298,298,0.0,1.0
3,3,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30,149,149,0.0,1.0
4,4,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41,30,149,149,0.0,1.0


Now we check that the msno's are truly unique

In [89]:
transactions_df_unique_msno.msno.value_counts()

MpEP5Cyd/sHg2FiZOVFAXCBrbDSYV7TDN9+4C+M6Fgk=    1
VmUI3TqkVUdCFYHSRQHh/w+vGv5kaMxcmhF4dYiHMD0=    1
LDUfQUOc6AzvzNoRfpGskZmLvZup9iKxLC0MKwxo7Ms=    1
Le9u7Um+p6axsSA/ZZqRw+1SOFMF3DeS8OFaQSFdY0I=    1
Zu3y6FKvN3BXwX0biDnhUjAmeLXgMEUgK7p2Cw7JQ6E=    1
                                               ..
dsmBThcF7iacu8wj8+r7is2oJfY13gqN55rLfqi8HKo=    1
zrSdeDn6ImgZ2svx3iwAUtAagGpyeWjtO2/VnbrgLK0=    1
Q3R4IR1WolJbsfBGx2VBVbhQTvdY8lcz1Zmri44G4fY=    1
Rs9UGfefAHRL/7xaG+uiLMH8qsH0LBdnUiWKPfYCETA=    1
wVoVYHpqgKq7ztnOYkCt2lsB+lUd22xqB1kvrR85eFY=    1
Name: msno, Length: 1197050, dtype: int64

## Transactions df feature engineering

In [129]:
# Add a feature for the average amount paid per day
transactions_df_unique_msno['avg_amount_paid_per_day'] = transactions_df_unique_msno.total_actual_amount_paid / transactions_df_unique_msno.total_payment_plan_days

In [130]:
# Feature for the difference between the listed price and the price paid by the customer.
transactions_df_unique_msno['diff_plan_list_price_actual_amount_paid'] = transactions_df_unique_msno.total_plan_list_price - transactions_df_unique_msno.total_actual_amount_paid

In [92]:
transactions_df_unique_msno.head()

,index,msno,most_frequent_payment_method_id,total_payment_plan_days,total_plan_list_price,total_actual_amount_paid,ratio_cancel,ratio_auto_renew,avg_amount_paid_per_day,diff_plan_list_price_actual_amount_paid
0,0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,22,395,1599,1599,0.0,0.0,4.048101,0
1,1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,0.0,1.0,3.300000,0
2,2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,60,298,298,0.0,1.0,4.966667,0
3,3,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30,149,149,0.0,1.0,4.966667,0
4,4,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41,30,149,149,0.0,1.0,4.966667,0


Now we will merge 'transactions_df_unique_msno' dataframe with the 'members_train' dataframe

In [131]:
members_train_trans_df = pd.merge(left=members_train_df, right=transactions_df_unique_msno, how='left', on='msno')

There are 35599 users who do not have transaction information. We will remove those users.

In [98]:
members_train_trans_df.isna().sum()

msno                                       0
is_churn                                   0
city                                       0
bd                                         0
gender                                     0
registered_via                             0
registration_init_time                     0
most_frequent_payment_method_id            0
total_payment_plan_days                    0
total_plan_list_price                      0
total_actual_amount_paid                   0
ratio_cancel                               0
ratio_auto_renew                           0
avg_amount_paid_per_day                    0
diff_plan_list_price_actual_amount_paid    0
dtype: int64

In [132]:
members_train_trans_df = members_train_trans_df.dropna(subset=['most_frequent_payment_method_id'])

In [133]:
# We will also delete the index column since it is an artifiact of pandas groupby
members_train_trans_df.drop('index', axis=1, inplace=True)

KeyError: "['index'] not found in axis"

### Merging the 'members_train_trans_df' dataframe with the user_logs dataframe
The user_logs dataframe contains information on the listening behaviours of users

We will merge the dataframes on the 'msno' column

The problem is that the user_logs dataframe contains many different entries for the same user. So, before we merge the dataframes we must make sure there is only one row per user in the user_logs dataframe

In [100]:
# More than one entry per user:
user_logs_df.msno.value_counts()

cm+LfkVpK6JtwxPGThgk6mtsvOrWBIIfGoft0H8u3Cc=    31
pxrkFAw4qeSGZt187lWd6LOmyM6AnBcnP6bx7RI5hC0=    31
9tDvsiMzH8fAIbGfkwBIWazeYVTFPsQe1Y2C6+gC4BM=    31
EcyNPbfT49APjaZlsbK+be9KvR72OpCnk0ok0PjHQVo=    31
Pc8vxTTR0/HBOKMKDGK0dMRh9VjKf+mb2GeGZDC8WmY=    31
                                                ..
/WyGNZrmNHylHjlC6ghF6uT/C/03uIzFKgteQhekvMk=     1
F6iY78a9q+D37pkVAs5tU3wBKTdYUita3l4bJAjqEJM=     1
WBnowJ0CeOxwfBaXLm6RYJa4h8z5cRC5hti9TbG7rZw=     1
wxMAMFV27loGgS3IyrcNOb04Un/0WG8yYy16wCnCfKE=     1
N7x6ynzlQ9opmZhED0ggWTfXLzSwmfocKZTPnVY8Z4Y=     1
Name: msno, Length: 1103894, dtype: int64

In [101]:
user_logs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18396362 entries, 0 to 18396361
Data columns (total 9 columns):
 #   Column      Dtype  
---  ------      -----  
 0   msno        object 
 1   date        int64  
 2   num_25      int64  
 3   num_50      int64  
 4   num_75      int64  
 5   num_985     int64  
 6   num_100     int64  
 7   num_unq     int64  
 8   total_secs  float64
dtypes: float64(1), int64(7), object(1)
memory usage: 1.2+ GB


We will create a dataframe to aggregate user_log information across so that each user has one entry

In [135]:
user_logs_unique_msno = pd.DataFrame()

In [136]:
# Create a avg_num_25 column
user_logs_unique_msno['avg_num_25'] = user_logs_df.groupby('msno')['num_25'].agg(np.mean)

In [137]:
# Create a avg_num_50 column
user_logs_unique_msno['avg_num_50'] = user_logs_df.groupby('msno')['num_50'].agg(np.mean)

In [138]:
# Create a avg_num_75 column
user_logs_unique_msno['avg_num_75'] = user_logs_df.groupby('msno')['num_75'].agg(np.mean)

In [139]:
# Create a avg_num_985 column
user_logs_unique_msno['avg_num_985'] = user_logs_df.groupby('msno')['num_985'].agg(np.mean)

In [140]:
# Create a avg_num_100 column
user_logs_unique_msno['avg_num_100'] = user_logs_df.groupby('msno')['num_100'].agg(np.mean)

In [141]:
# Create a avg_num_unq column
user_logs_unique_msno['avg_num_unq'] = user_logs_df.groupby('msno')['num_unq'].agg(np.mean)

In [142]:
# Create a avg_total_secs column
user_logs_unique_msno['avg_total_secs'] = user_logs_df.groupby('msno')['total_secs'].agg(np.mean)

In [143]:
user_logs_unique_msno.reset_index(inplace=True)

In [112]:
user_logs_unique_msno.head()

,msno,avg_num_25,avg_num_50,avg_num_75,avg_num_985,avg_num_100,avg_num_unq,avg_total_secs
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,3.307692,0.423077,0.384615,0.192308,18.153846,20.384615,4534.900962
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,6.161290,2.903226,2.419355,4.645161,19.000000,28.548387,6210.577161
2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,1.535714,0.428571,0.535714,0.428571,17.321429,16.714286,4121.830714
3,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,9.857143,7.761905,4.761905,3.047619,20.761905,39.428571,7137.931333
4,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,3.620690,0.827586,1.344828,1.206897,16.517241,7.931034,4014.939552


Now merge 'user_logs_unique_msno' with 'members_train_trans_df' to create the final dataframe

In [144]:
aggregated_df = pd.merge(left=members_train_trans_df, right=user_logs_unique_msno, how='left', on='msno')

Now we will remove the users with no user log information

In [ ]:
aggregated_df.isna().sum()

msno                                       0
is_churn                                   0
city                                       0
bd                                         0
gender                                     0
registered_via                             0
registration_init_time                     0
most_frequent_payment_method_id            0
total_payment_plan_days                    0
total_plan_list_price                      0
total_actual_amount_paid                   0
ratio_cancel                               0
ratio_auto_renew                           0
avg_amount_paid_per_day                    0
diff_plan_list_price_actual_amount_paid    0
avg_num_25                                 0
avg_num_50                                 0
avg_num_75                                 0
avg_num_985                                0
avg_num_100                                0
avg_num_unq                                0
avg_total_secs                             0
dtype: int

In [145]:
aggregated_df = aggregated_df.dropna(subset=['avg_num_25'])

# EDA

In [119]:
# eda tings

Remove unnecessary features

In [147]:
# We will remove registration_init_time since it is difficult to use it in training
aggregated_df.drop('registration_init_time', axis=1, inplace=True)

Convert floats to ints

In [152]:
aggregated_df = aggregated_df.astype({'city' : int, 'bd' : int, 'registered_via' : int, 'most_frequent_payment_method_id' : int, 'total_payment_plan_days' : int, 'total_plan_list_price' : int, 'total_actual_amount_paid' : int, 'diff_plan_list_price_actual_amount_paid' : int})

# Feature Scaling

In [153]:
aggregated_df.head()

,msno,is_churn,city,bd,gender,registered_via,most_frequent_payment_method_id,total_payment_plan_days,total_plan_list_price,total_actual_amount_paid,...,ratio_auto_renew,avg_amount_paid_per_day,diff_plan_list_price_actual_amount_paid,avg_num_25,avg_num_50,avg_num_75,avg_num_985,avg_num_100,avg_num_unq,avg_total_secs
0,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,13,20,male,3,36,30,180,180,...,0.0,6.000000,0,0.000000,0.666667,0.333333,0.000000,4.333333,5.000000,1164.418167
1,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,13,18,male,3,15,150,300,300,...,0.0,2.000000,0,11.950000,2.850000,1.600000,1.100000,10.250000,21.600000,3390.523350
3,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,13,35,female,7,41,240,792,792,...,1.0,3.300000,0,0.600000,0.466667,0.266667,0.266667,64.133333,36.533333,15992.149400
4,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,1,22,0,unspecified,3,40,30,149,149,...,1.0,4.966667,0,1.500000,0.250000,0.250000,1.000000,9.250000,12.000000,2527.666250
6,moRTKhKIDvb+C8ZHOgmaF4dXMLk0jOn65d7a8tQ2Eds=,1,9,28,female,3,38,410,1788,1788,...,0.0,4.360976,0,2.516129,0.645161,0.967742,0.516129,33.000000,24.451613,9381.670806


In [163]:
aggregated_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 725722 entries, 0 to 825367
Data columns (total 21 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   msno                                     725722 non-null  object 
 1   is_churn                                 725722 non-null  int64  
 2   city                                     725722 non-null  int32  
 3   bd                                       725722 non-null  int32  
 4   gender                                   725722 non-null  object 
 5   registered_via                           725722 non-null  int32  
 6   most_frequent_payment_method_id          725722 non-null  int32  
 7   total_payment_plan_days                  725722 non-null  int32  
 8   total_plan_list_price                    725722 non-null  int32  
 9   total_actual_amount_paid                 725722 non-null  int32  
 10  ratio_cancel                    

Perform one hot encoding for the gender feature

In [207]:
from sklearn.preprocessing import OneHotEncoder

In [208]:
enc = OneHotEncoder(handle_unknown='ignore')

In [209]:
enc_df = pd.DataFrame(enc.fit_transform(aggregated_df[['gender']]).toarray())

KeyError: "None of [Index(['gender'], dtype='object')] are in the [columns]"

In [188]:
enc.categories_

[array(['female', 'male', 'unspecified'], dtype=object)]

In [196]:
enc_df.rename(columns={0 : 'female', 1 : 'male', 2 : 'unspecified'}, inplace=True)

In [197]:
aggregated_df = aggregated_df.join(enc_df)

In [200]:
aggregated_df.head()

,msno,is_churn,city,bd,registered_via,most_frequent_payment_method_id,total_payment_plan_days,total_plan_list_price,total_actual_amount_paid,ratio_cancel,...,avg_num_985,avg_num_100,avg_num_unq,avg_total_secs,0,1,2,female,male,unspecified
0,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,13,20,3,36,30,180,180,0.000,...,0.000000,4.333333,5.000000,1164.418167,0.0,1.0,0.0,0.0,1.0,0.0
1,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,13,18,3,15,150,300,300,0.000,...,1.100000,10.250000,21.600000,3390.523350,0.0,1.0,0.0,0.0,1.0,0.0
3,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,13,35,7,41,240,792,792,0.125,...,0.266667,64.133333,36.533333,15992.149400,0.0,0.0,1.0,0.0,0.0,1.0
4,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,1,22,0,3,40,30,149,149,1.000,...,1.000000,9.250000,12.000000,2527.666250,1.0,0.0,0.0,1.0,0.0,0.0
6,moRTKhKIDvb+C8ZHOgmaF4dXMLk0jOn65d7a8tQ2Eds=,1,9,28,3,38,410,1788,1788,0.000,...,0.516129,33.000000,24.451613,9381.670806,0.0,0.0,1.0,0.0,0.0,1.0


In [210]:
aggregated_df.drop([0, 1, 2, 'female', 'male', 'unspecified'], axis = 1, inplace=True)

In [211]:
from sklearn.model_selection import train_test_split

In [212]:
X_train, X_test, y_train, y_test = train_test_split(aggregated_df.drop(['msno', 'is_churn'], axis=1), aggregated_df['is_churn'], test_size=0.2, random_state=42)

# Machine Learning!

In [213]:
X_train.isna().sum()

city                                       0
bd                                         0
registered_via                             0
most_frequent_payment_method_id            0
total_payment_plan_days                    0
total_plan_list_price                      0
total_actual_amount_paid                   0
ratio_cancel                               0
ratio_auto_renew                           0
avg_amount_paid_per_day                    0
diff_plan_list_price_actual_amount_paid    0
avg_num_25                                 0
avg_num_50                                 0
avg_num_75                                 0
avg_num_985                                0
avg_num_100                                0
avg_num_unq                                0
avg_total_secs                             0
dtype: int64

In [217]:

from sklearn.ensemble import RandomForestClassifier

In [218]:
clf_rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)

In [219]:
clf_rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, random_state=42)

In [220]:
y_train_pred = clf_rf.predict(X_train)

In [226]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [222]:
accuracy_score(y_train, y_train_pred)

0.9708617461594241

In [224]:
y_test_pred = clf_rf.predict(X_test)

In [225]:
accuracy_score(y_test, y_test_pred)

0.9714423507526956

In [227]:
precision_score(y_test, y_test_pred)

0.9229993275050438

In [228]:
recall_score(y_test, y_test_pred)

0.5982347172278523